# Strategies

> Fill in a module description here

In [ ]:
#| default_exp strategies

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
%load_ext autoreload


In [ ]:
%autoreload 2

In [ ]:
#| export
import clue.model as model
import clue.ui as ui
import clue.board as bd
from random import choice,shuffle,choices
import os

In [ ]:
#| hide


sboard = bd.Board()
sState = sboard.updateState(4)


In [ ]:
#| export
def getPlay(state):
    state.board.show(state)
    l = state.board.pickRoom(state)
    p, w = ui.choosePandW(state)
    return p-1, w-1, l



In [ ]:

print(ui.showChoice(getPlay(sState)))

                    Lounge                                      
                      🧐                                        
                       ▒▒▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒                                      
                       ▒░▒

Choose from 1 to 3:  3


    People              Weapons
--  ------------------  -------------
 1  👠 Miss Scarlet     🔪 Dagger
 2  🧐 Colonel Mustard  *🔫 Revolver
 3  *🥼  Mrs. White     *🪢 Rope
 4  🐉 Mr. Green        🔧 Wrench
 5  🍆 Professor Plum   🕯️ Candlestick
 6  *🦚 Mrs. Peacock    🪠 Lead Pipe


 4 5


🐉 Mr. Green with the 🕯️ Candlestick in the Lounge


In [ ]:
#| hide
#stguess = getPlay(1,stgame,stmoves)
#print(ui.showChoice(stguess))
def seenCounts(state):
    knoweledge = ui.getSeen(state)
    p = [0,0,0]
    for x in knoweledge:
        i = int(x/100) -1
        p[i] += 1

    return p

In [ ]:
#| hide
#stguess = getPlay(2,stgame,stmoves)

In [ ]:
#| export




def simpleGuess(state):
   


    playerRoom = model.roomOf(pc,state)
    exclude  = set(model.getSeen(state))
    
    pdeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  pdeck if x not in exclude]
    p = choice(pdeck)%100
    
    pdeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in pdeck if x not in exclude]
    w = choice(pdeck)%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)


In [ ]:
#| export

def itemsForType(t):
    if t == 1:
        return model.people
    if t == 2:
        return model.weapons
    return model.rooms
        
def itemGuess(t,moves):
    l = itemsForType(t)

    ret = [1 for x in range(len(l))]
    for x in moves:
        ret[x[t-1]] += 1
    #print(ret)
    return ret
    
def showCounts(t,moves):
    items = itemGuess(t,moves)
    base = itemsForType(t)
    #print(items,base)
    for i in range(len(items)):
        print(i,ui.cardFull(base[i]),items[i])


                              




In [ ]:
#| export
def biasWalk(state):
    
    def bWalk(exclude,state):

        playerRoom = model.roomOf(state.player,state) 
        dest = model.roomsAround(playerRoom)
        if model.keyCard(3,dest[a]) not in exclude:
            return dest[a]%100
        if model.keyCard(3,dest[b]) not in exclude:
            return dest[b]%100
        
        return dest[c]%100
    return bWalk
    
    
 

In [ ]:
#| export
def rWalk(exclude,state):

    playerRoom = model.roomOf(state.player,state)
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom,state)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
                              
    return l

In [ ]:
#| export


def roomsAway(roomNum,roomSize):
    foo = [x for x in range(roomSize)]
    middle = int( roomSize / 2)
    if roomNum <middle:
        lm = foo[roomNum - int(middle):]
        lSide = lm + foo[:roomNum] 
        lSide.reverse()
        rSide = foo[roomNum + 1:roomNum + 1 + middle]
    elif roomNum > middle:
        lSide = foo[roomNum-middle:roomNum]
        lSide.reverse()
        rSide = foo[roomNum +1: ] + foo[:roomNum-middle]
    else:
        lSide = foo[:middle]
        lSide.reverse()
        rSide = foo[middle+1:]
    return lSide, rSide

def repelWalk(exclude,state):
    playerRoom = model.roomOf(state.player,state)
    lRooms, rRooms = roomsAway(playerRoom,len(model.rooms))
    #print("staring at " + ui.cardFull(model.rooms[playerRoom]))
    #print(playerRoom,lRooms,rRooms)
    if not (model.keyCard(3,playerRoom) in exclude):
     return playerRoom
    #print(exclude)
    def score(rooms):
        rooms.reverse()
        weights = []
        cards = []
        for i in range(len(rooms)):
            r = rooms[i]
            cards.append(str(model.keyCard(3,r)) + " " + model.rooms[r][0])
            
            if model.keyCard(3,r) in exclude:
                w = 2**i
                #print(i,w)
                weights.append(w)
            else:
                weights.append(0)
        #print(cards,weights)
        return sum(weights)


    lPull = score(lRooms)
    rPull = score(rRooms)
    #trace.append([playerRoom,lPull,rPull])
    #print(trace)
    #ui.showBoard(pc,game,moves)
    achoice = list(model.roomsAround(playerRoom,state))
    achoice.sort()
    if lPull > rPull:
        #print("to" + ui.cardFull(model.rooms[achoice[2]]))
        return achoice[-1]
    if lPull < rPull:
        
        #print("to" + ui.cardFull(model.rooms[achoice[0]]))
        return achoice[0]
    #print("same")    
    return rWalk(exclude,state)


In [ ]:
#| export
def mimic(memory,walk):
    def betterGuess(state):

        exclude  = set(model.getSeen(state))

        if memory < 0:
            examine = []
        else:
            examine = state.moves[-memory:]


        keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
        pdeck = [x for x in  keydeck if x not in exclude]
        pC = itemGuess(1,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        p = choices(pdeck,weights=pWeights)
        p = p[0]%100

        keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
        pdeck = [x for x in keydeck if x not in exclude]
        pC = itemGuess(2,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        w = choices(pdeck,weights=pWeights)[0]%100
        
        l = walk(exclude,state)

        return (p,w,l)
    
    return betterGuess

In [ ]:
#| export
def cheater(walk):
    def betterGuess(state):

 
        exclude  = set(model.handOf(state) +[x[-1] for x in state.moves] )

       
        examine = []
   


        keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
        pdeck = [x for x in  keydeck if x not in exclude]
        pC = itemGuess(1,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        p = choices(pdeck,weights=pWeights)
        p = p[0]%100

        keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
        pdeck = [x for x in keydeck if x not in exclude]
        pC = itemGuess(2,examine)
        pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]

        w = choices(pdeck,weights=pWeights)[0]%100
        
        l = walk(exclude,state)

        return (p,w,l)
    return betterGuess


def halfRight(walk):
    oracle = cheater(walk)
    
    def betterGuess(state):
        p,w,l = oracle(state)
        exclude  = model.handOf(state) +[x[-1] for x in state.moves] 
        exclude = [x for x in exclude if x < 300]
        

        if len(exclude) <= 0:
            return (p,w,l)

        wrong = choice(exclude)
        if wrong < 200:
            p = wrong%100
        else:
            w = wrong%100

        return (p,w,l)
    return betterGuess

In [ ]:
#| export

def hasGuess(guess,moves):
    p, w, l = guess[:3]
    c = (p,w,l)
    for x in moves:
        p, w, l = x[:3]
        xc = (p,w,l)
        if c == xc:
            return True
    return False

def repeater(strats,verbose=False):
    def betterGuess(state):
        counter = 0
        for strat in strats:
            
            g =  strat(state)
            if not hasGuess(g,state.moves):
                if verbose and counter > 0:
                    print("counter " + str(counter))
                return g
            counter += 1
                        
        if verbose and counter > 0:
            print(" over counter " + str(counter))
        return strats[-1](state)
                          
    return betterGuess

In [ ]:
r1 = repeater([mimic(1*(x+3),rWalk) for x in range(4)])
r1(sState)

(3, 5, 6)

In [ ]:
r2 = repeater([mimic(1*(x+3),repelWalk) for x in range(4)])
r2(sState)

(1, 5, 7)

In [ ]:
r3 = repeater([halfRight(rWalk) for x in range(4)])
r3(sState)

(4, 2, 6)